In [2]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, BertTokenizer, BertForMaskedLM

def gpt2_generate_response(input_text):
    tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
    model = GPT2LMHeadModel.from_pretrained('gpt2')
    input_ids = tokenizer.encode(input_text, return_tensors="pt")
    output = model.generate(input_ids, max_length=100, num_return_sequences=1)
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    return generated_text

def bert_generate_text(input_text):
    tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
    model = BertForMaskedLM.from_pretrained("bert-base-uncased")
    input_ids = tokenizer.encode(input_text, return_tensors="pt")
    output = model.generate(input_ids, max_length=100, num_return_sequences=1, pad_token_id=tokenizer.eos_token_id)
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    return generated_text


user_input = 'Hello, how are you?'
response = bert_generate_text(user_input)
print(response)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


hello, how are you? hello? hello? hello? hello? hello? hello? hello? hello? hello? hello? hello? hello? hello? hello? hello? hello? hello? hello? hello? hello! hello! hello! hello! hello! hello!!!!!!!!! hello!!! hello! hello!!!!!!!!!!!!!!! hello!!!!!!!!!!!


In [6]:
from transformers import BertTokenizer, BertForMaskedLM
import torch

def predict_masked_word(sentence):
    # Load pre-trained BERT model and tokenizer
    tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
    model = BertForMaskedLM.from_pretrained("bert-base-uncased")
    
    # Tokenize input sentence and find the index of the masked token
    input_ids = tokenizer.encode(sentence, return_tensors="pt")
    mask_token_index = torch.where(input_ids == tokenizer.mask_token_id)[1]
    
    # Predict the masked token
    with torch.no_grad():
        output = model(input_ids)
    
    # Get the logits for the masked token and find the top prediction
    mask_token_logits = output.logits[0, mask_token_index, :]
    top_token_id = torch.argmax(mask_token_logits, dim=-1)
    
    # Decode the predicted token ID back to a word
    predicted_token = tokenizer.decode(top_token_id)
    return predicted_token

sentence = "Once upon a time, there was person [MASK]?"
predicted_word = predict_masked_word(sentence)
print(f"Predicted word: {predicted_word}")


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Predicted word: ##hood


In [7]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

def gpt2_generate_response(input_text):
    
    tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
    model = GPT2LMHeadModel.from_pretrained('gpt2')
    
    input_ids = tokenizer.encode(input_text, return_tensors="pt")
    
    output = model.generate(input_ids, max_length=65, num_return_sequences=1)
    
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    
    return generated_text

input_text = "There was a time"
generated_text = gpt2_generate_response(input_text)
print(generated_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


There was a time when the world was still a little bit more open to the idea of a world where people could live in peace and harmony.

"But now, we have a world where people are not afraid to be themselves. We have a world where people are not afraid to be themselves. We have a world


In [9]:
# ---------------------- Fine Tuning ----------------------

from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch
from torch.utils.data import DataLoader, Dataset

# Load pre-trained model and tokenizer
model = GPT2LMHeadModel.from_pretrained("gpt2")
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

class TextDataset(Dataset):
    def __init__(self, texts, tokenizer, max_length=1024):
        self.tokenizer = tokenizer
        self.input_ids = []
        
        for text in texts:
            tokenized_text = tokenizer.encode(text, truncation=True, max_length=max_length)
            self.input_ids.append(tokenized_text)

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return torch.tensor(self.input_ids[idx])

# Example input texts (split your data into smaller chunks if necessary)
texts = [
    "Your very long input text here. " * 100,  # Repeat to simulate a long text
    "Another example of a long input text." * 100
]

# Create dataset and dataloader
dataset = TextDataset(texts, tokenizer)
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)

# Fine-tuning parameters
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)
epochs = 3

# Fine-tuning loop
for epoch in range(epochs):
    model.train()
    for batch in dataloader:
        optimizer.zero_grad()
        input_ids = batch.squeeze(0)  # Remove batch dimension
        outputs = model(input_ids, labels=input_ids)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch+1}: Loss - {loss.item()}")

# Save fine-tuned model
model.save_pretrained("fine_tuned_gpt2")
tokenizer.save_pretrained("fine_tuned_gpt2")


In [10]:
finetuned_model = GPT2LMHeadModel.from_pretrained("./fine_tuned_gpt2")
finetuned_tokenizer = GPT2Tokenizer.from_pretrained("./fine_tuned_gpt2")

input_text = "There was a time"
input_ids = finetuned_tokenizer.encode(input_text, return_tensors='pt')
output = finetuned_model.generate(input_ids, max_length=50, num_return_sequences=1)

print(finetuned_tokenizer.decode(output[0], skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


There was a time when I was a little bit of a nerd. I was a little bit of a nerd. I was a little bit of a nerd. I was a little bit of a nerd. I was a little bit of a nerd.
